In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("Fast Food Analysis Project Spreadsheet.xlsx")

In [ ]:
df.head()

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
print(df.columns)

In [ ]:
df = df.rename(columns={'Restaurant': 'restaurant_name', 'Item Name': 'item_name', 'Calories ': 'calories', 'Protein (g)': 'protein_g',
 'Total Fats (g)': 'fats_g', 'Sodium (mg)': 'sodium_mg', 'Total Sugars (g)': 'sugars_g',
 'Cholestrol (mg)': 'cholestrol_mg', 'Price ($)': 'price', 'Carbs (g)': 'carbs_g', 
 'Fiber (g)': 'fiber_g', 'Item Type': 'item_type'})
df.head()

In [ ]:
df.isnull().sum()


In [ ]:
df.duplicated().sum()

In [ ]:
## starting tier one analysis: basic averages, ranks of each restaurant and item with the highest calories, protein etc. 
df.describe()

In [ ]:
## gonna use a loop to find top and bottom 10 of each metric (protein, fiber etc)
metrics = ['calories', 'protein_g', 'fats_g', 'sodium_mg', 'cholestrol_mg', 'price', 'carbs_g', 'fiber_g']

for metric in metrics:
    print(f"Top 10 {metric.title()}")
    display(df.sort_values(by=metric, ascending=False).head(10))

    print(f"Bottom 10 {metric.title()}")
    display(df.sort_values(by=metric, ascending=False).tail(10))

In [ ]:
#now gonna do a loop for the averages (gonna use medians, chose due to discovery of outliers) based on restaurants
for metric in metrics:
    print(f"Restaurants Ranked by Median {metric.title()} (High to Low)")

    ranked = df.groupby('restaurant_name')[metric].median().sort_values(ascending=False)
    display(ranked)

In [ ]:
#do the same as above but by item type now
for metric in metrics:
    print(f"Item Types Ranked by Median {metric.title()} (High to Low)")

    ranked = df.groupby('item_type')[metric].median().sort_values(ascending=False)
    display(ranked)

In [ ]:
## starting tier two analysis
## start of the clustering technique 
df.describe()


In [ ]:
## making definitions of clusters + finding how many there are; will in next iteration of project separate by item type! ##
cluster_rules = {
    'high_protein_low_fat': (df['protein_g'] >= 20) & (df['fats_g'] <= 23),
    'low_carb_high_fat': (df['carbs_g'] <= 40) & (df['fats_g'] >= 23),
    'high_fiber_low_sodium': (df['fiber_g'] >= 3) & (df['sodium_mg'] <= 1015),
    'high_protein_low_price': (df['protein_g'] >= 20) & (df['price'] <= 6.60), 
    'low_calorie_high_fiber': (df['calories'] <= 440) & (df['fiber_g'] >= 3)
}

for name, rule in cluster_rules.items():
    df[f"cluster_{name}"] = rule

In [ ]:
for name in cluster_rules:
    print(f"{name}: {df[f'cluster_{name}'].sum()} items")

In [ ]:
cluster_name_map = {
    'high_protein_low_fat': 'High Protein + Low Fat', 
    'low_carb_high_fat': 'Low Carb + High Fat', 
    'high_fiber_low_sodium': 'High Fiber + Low Sodium', 
    'high_protein_low_price': 'High Protein + Low Price', 
    'low_calorie_high_fiber': 'Low Calorie + High Fiber'
}

df['manual_cluster'] = 'None'

for name in cluster_rules:
    clean_label = cluster_name_map[name]
    df.loc[df[f'cluster_{name}'], 'manual_cluster'] = clean_label

In [ ]:
df.head()

In [ ]:
df[df['manual_cluster'] != 'None'].sort_values('manual_cluster')

In [ ]:
## for easier viewing of the items and their clusters ##
cluster_cols = [col for col in df.columns if col.startswith('cluster_')]

df_display = df.drop(columns=cluster_cols)

In [ ]:
df_display.head()

In [ ]:
df[df['manual_cluster'] == 'High Fiber + Low Sodium']

In [ ]:
#now gonna do a loop to rank items in each cluster by metrics
for metric in metrics:
    for cluster in df['manual_cluster'].unique():
        if cluster != 'None':
            cluster_df = df_display[df['manual_cluster'] == cluster]
            sorted_df = cluster_df.sort_values(by=metric, ascending=False)

            print(f"Ranked Items in Cluster: {cluster} by {metric.title()}")
            display(sorted_df)

In [ ]:
## from this code, we can see if items are matching for multiple clusters
df[[col for col in df.columns if col.startswith('cluster_')]].sum(axis=1).value_counts()

In [ ]:
df_display.head()